In [4]:
import json
import time
import random
import re
import os
from datetime import datetime
from typing import Optional
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


class GearVNCrawlerPro:

    ALL_CATEGORIES = [
        ("collections/laptop-gaming-ban-chay", "Laptop Gaming"),
        ("collections/laptop-van-phong", "Laptop Van Phong"),
        ("collections/pc-gvn", "PC GVN"),
        ("collections/man-hinh-may-tinh", "Man Hinh"),
        ("collections/ban-phim-may-tinh", "Ban Phim"),
        ("collections/chuot-may-tinh", "Chuot"),
        ("collections/tai-nghe-may-tinh", "Tai Nghe"),
        ("collections/ghe-gia-tot", "Ghe Gaming"),
        ("collections/vga-card-man-hinh", "VGA Card"),
        ("collections/cpu-bo-vi-xu-ly", "CPU"),
        ("collections/mainboard-bo-mach-chu", "Mainboard"),
        ("collections/ram-pc", "RAM"),
        ("collections/o-cung-ssd", "O Cung SSD"),
        ("collections/case-may-tinh", "Case"),
        ("collections/nguon-may-tinh", "Nguon"),
        ("collections/tan-nhiet-khi", "Tan Nhiet"),
        ("collections/webcam", "Webcam"),
        ("collections/loa-may-tinh", "Loa"),
        ("collections/phu-kien", "Phu Kien"),
        ("collections/may-choi-game", "Console/Handheld"),
    ]
    
    def __init__(self, base_url: str = "https://gearvn.com", headless: bool = True):
        self.base_url = base_url
        self.headless = headless
        self.driver = None
        self.crawled_urls: set = set()
        self.total_crawled: int = 0
        self.session_start = datetime.now()
        
    def _create_driver(self):
        """Create WebDriver with optimized configuration."""
        chrome_options = Options()
        
        if self.headless:
            chrome_options.add_argument("--headless=new")
        
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--disable-blink-features=AutomationControlled")
        chrome_options.add_argument("--window-size=1920,1080")
        chrome_options.add_argument("--disable-images")
        chrome_options.add_argument(
            "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        )
        chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        chrome_options.add_experimental_option('useAutomationExtension', False)
        chrome_options.page_load_strategy = 'eager'
        chrome_options.add_argument("--log-level=3")
        chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
        
        driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()), 
            options=chrome_options
        )
        driver.execute_script(
            "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
        )
        driver.set_page_load_timeout(30)
        return driver
    
    def _get_driver(self):
        """Get or create driver (reuse existing)."""
        if self.driver is None:
            self.driver = self._create_driver()
        return self.driver
    
    def _close_driver(self):
        """Close the driver."""
        if self.driver:
            try:
                self.driver.quit()
            except Exception:
                pass
            self.driver = None
    
    def _scroll_to_load_all(self, max_scrolls: int = 15, pause_time: float = 1.0):
        """Scroll page to load all products (lazy loading)."""
        driver = self._get_driver()
        last_height = driver.execute_script("return document.body.scrollHeight")
        scrolls = 0
        
        while scrolls < max_scrolls:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(pause_time)
            new_height = driver.execute_script("return document.body.scrollHeight")
            
            if new_height == last_height:
                time.sleep(0.5)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(pause_time)
                final_height = driver.execute_script("return document.body.scrollHeight")
                if final_height == new_height:
                    break
            
            last_height = new_height
            scrolls += 1
        
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(0.3)
    
    def get_all_product_links_from_category(
        self, 
        category_slug: str, 
        category_name: str, 
        max_pages: int = 20
    ) -> list:
        """Get ALL product links from a category (supports pagination)."""
        driver = self._get_driver()
        all_links = []
        page = 1
        
        while page <= max_pages:
            try:
                url = (
                    f"{self.base_url}/{category_slug}" 
                    if page == 1 
                    else f"{self.base_url}/{category_slug}?page={page}"
                )
                
                print(f"  [Page {page}] {url}")
                driver.get(url)
                
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.TAG_NAME, "body"))
                )
                time.sleep(1.5)
                
                self._scroll_to_load_all(max_scrolls=10, pause_time=0.8)
                
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                
                product_links_on_page = set()
                for a_tag in soup.select('a[href*="/products/"]'):
                    href = a_tag.get('href')
                    if href and isinstance(href, str) and '/products/' in href:
                        full_link = (
                            self.base_url + href 
                            if not href.startswith('http') 
                            else href
                        )
                        excluded_patterns = ['/cart', '/account', '/pages', '#']
                        if not any(x in full_link for x in excluded_patterns):
                            full_link = full_link.split('?')[0]
                            if full_link not in self.crawled_urls:
                                product_links_on_page.add(full_link)
                
                if not product_links_on_page:
                    print(f"  [!] No new products found on page {page}, stopping pagination")
                    break
                
                for link in product_links_on_page:
                    all_links.append({'url': link, 'category': category_name})
                
                print(f"     [+] Found {len(product_links_on_page)} new products")
                
                next_page = soup.select_one(
                    'a.next, a[rel="next"], .pagination a:contains("Next"), '
                    '.pagination a:contains(">")'
                )
                if not next_page and page > 1:
                    pagination = soup.select('.pagination a, .paginate a')
                    has_more = any(str(page + 1) in (p.text or '') for p in pagination)
                    if not has_more:
                        print(f"  [*] Reached last page ({page} pages)")
                        break
                
                page += 1
                time.sleep(random.uniform(0.5, 1.0))
                
            except Exception as e:
                print(f"  [ERROR] Page {page}: {e}")
                break
        
        print(f"  [TOTAL] {len(all_links)} product links from {category_name}")
        return all_links
    
    def parse_product_detail(self, product_info: dict) -> Optional[dict]:
        """Extract detailed product information."""
        url = product_info['url']
        category = product_info['category']
        
        if url in self.crawled_urls:
            return None
        
        driver = self._get_driver()
        
        try:
            driver.get(url)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "h1"))
            )
            time.sleep(0.8)
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Product name
            name_tag = soup.select_one('h1.product_name, h1.title-product, h1')
            name = name_tag.text.strip() if name_tag else "Unknown"
            
            # Price
            price = 0
            original_price = 0
            discount = 0
            
            price_selectors = [
                '.product_price .price-box-price',
                '.special-price',
                '.price .price-box-price', 
                'span[class*="price"]',
                '.product_price'
            ]
            for sel in price_selectors:
                price_tag = soup.select_one(sel)
                if price_tag:
                    price_str = price_tag.text.strip()
                    digits = ''.join(filter(str.isdigit, price_str))
                    if digits:
                        price = int(digits)
                        break
            
            # Original price
            original_price_tag = soup.select_one('.price-box-price-old, .old-price, del')
            if original_price_tag:
                orig_str = original_price_tag.text.strip()
                orig_digits = ''.join(filter(str.isdigit, orig_str))
                if orig_digits:
                    original_price = int(orig_digits)
                    if original_price > price > 0:
                        discount = round((1 - price / original_price) * 100)
            
            # Specifications
            specs = {}
            spec_tables = soup.select(
                '.technical-content tr, .so-luoc-phan-cung tr, '
                '.product-specs tr, table tr'
            )
            for row in spec_tables:
                cols = row.find_all(['td', 'th'])
                if len(cols) >= 2:
                    key = cols[0].text.strip()
                    value = cols[1].text.strip()
                    if key and value and len(key) < 100:
                        specs[key] = value
            
            # Images
            images = []
            image_selectors = [
                '.product-gallery img', 
                '.product-image img', 
                'img[src*="product"]'
            ]
            for sel in image_selectors:
                img_tags = soup.select(sel)
                for img in img_tags[:5]:
                    src = img.get('src') or img.get('data-src') or img.get('data-lazy')
                    if src and isinstance(src, str) and 'http' in src and src not in images:
                        images.append(src)
                if images:
                    break
            
            # Description
            desc_tag = soup.select_one('.product-description, .product-content, article')
            description = (
                desc_tag.text.strip()[:500] 
                if desc_tag 
                else f"{name} - Price: {price:,}d"
            )
            
            # Rating
            rating = 0.0
            rating_tag = soup.select_one('.rating-result, .star-rating, [class*="rating"]')
            if rating_tag:
                rating_match = re.search(r'(\d+\.?\d*)', rating_tag.text.strip())
                if rating_match:
                    rating = float(rating_match.group(1))
            
            # Stock status
            in_stock = not bool(
                soup.select_one('.out-of-stock, .het-hang, [class*="soldout"]')
            )
            
            # Brand
            brand = ""
            brand_tag = soup.select_one('.product-brand, .vendor, [class*="brand"]')
            if brand_tag:
                brand = brand_tag.text.strip()
            else:
                known_brands = [
                    'ASUS', 'MSI', 'Lenovo', 'Acer', 'HP', 'Dell', 'Gigabyte',
                    'Razer', 'Logitech', 'Corsair', 'HyperX', 'SteelSeries',
                    'Samsung', 'LG', 'BenQ', 'ViewSonic', 'AOC', 'Intel', 'AMD'
                ]
                for b in known_brands:
                    if b.lower() in name.lower():
                        brand = b
                        break
            
            self.crawled_urls.add(url)
            self.total_crawled += 1
            
            return {
                "name": name,
                "price": price,
                "original_price": original_price,
                "discount_percent": discount,
                "url": url,
                "category": category,
                "brand": brand,
                "specs": specs,
                "description": description,
                "images": images,
                "rating": rating,
                "in_stock": in_stock,
                "crawled_at": datetime.now().isoformat()
            }
            
        except Exception as e:
            print(f"      [!] Error: {str(e)[:50]}")
            return None
    
    def save_batch(
        self, 
        products: list, 
        filename: str = "crawled_products.json", 
        append: bool = True
    ):
        """Save data in batches (supports append mode)."""
        if not products:
            return
        
        existing_data = []
        existing_urls = set()
        
        if append and os.path.exists(filename):
            try:
                with open(filename, 'r', encoding='utf-8') as f:
                    existing_data = json.load(f)
                    existing_urls = {p['url'] for p in existing_data}
            except Exception:
                pass
        
        new_products = [p for p in products if p['url'] not in existing_urls]
        all_data = existing_data + new_products
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(all_data, f, ensure_ascii=False, indent=2)
        
        print(f"[SAVED] {len(new_products)} new products (Total: {len(all_data)})")
    
    def crawl_category_full(
        self, 
        category_slug: str, 
        category_name: str, 
        save_every: int = 20, 
        filename: str = "crawled_products.json"
    ) -> list:
        """Crawl ALL products from a category."""
        print(f"\n{'=' * 60}")
        print(f"CRAWL CATEGORY: {category_name}")
        print('=' * 60)
        
        product_links = self.get_all_product_links_from_category(
            category_slug, category_name
        )
        
        if not product_links:
            print("[!] No products found")
            return []
        
        products = []
        batch = []
        
        for i, product_info in enumerate(product_links):
            print(f"  [{i + 1}/{len(product_links)}] {product_info['url'].split('/')[-1][:40]}...")
            
            product = self.parse_product_detail(product_info)
            if product:
                products.append(product)
                batch.append(product)
                print(f"      [+] {product['name'][:35]}... | {product['price']:,}d")
            
            if len(batch) >= save_every:
                self.save_batch(batch, filename, append=True)
                batch = []
            
            time.sleep(random.uniform(0.3, 0.8))
        
        if batch:
            self.save_batch(batch, filename, append=True)
        
        print(f"\n[{category_name}] Completed: {len(products)} products")
        return products
    
    def crawl_all(
        self, 
        categories: Optional[list] = None, 
        save_every: int = 20, 
        filename: str = "crawled_products.json"
    ) -> list:
        """
        Crawl ALL products from all categories.
        
        Args:
            categories: List of categories (None = all)
            save_every: Save after every N products
            filename: Output filename
        """
        categories_to_crawl = categories if categories else self.ALL_CATEGORIES
        
        print("\n" + "=" * 60)
        print("GEARVN CRAWLER PRO - CRAWL ALL PRODUCTS")
        print("=" * 60)
        print(f"\nCategories: {len(categories_to_crawl)}")
        print(f"Output file: {filename}")
        print(f"Started: {self.session_start.strftime('%Y-%m-%d %H:%M:%S')}")
        print("=" * 60 + "\n")
        
        all_products = []
        
        try:
            for idx, (category_slug, category_name) in enumerate(categories_to_crawl):
                print(f"\n[{idx + 1}/{len(categories_to_crawl)}] {category_name}")
                
                category_products = self.crawl_category_full(
                    category_slug, category_name,
                    save_every=save_every,
                    filename=filename
                )
                all_products.extend(category_products)
                
                time.sleep(random.uniform(1, 2))
                
        except KeyboardInterrupt:
            print("\n\n[!] User interrupted crawler")
        finally:
            self._close_driver()
        
        elapsed = datetime.now() - self.session_start
        
        print("\n" + "=" * 60)
        print("CRAWL COMPLETED")
        print("=" * 60)
        print(f"Total products crawled: {len(all_products)}")
        print(f"Elapsed time: {elapsed}")
        print(f"Output file: {filename}")
        
        return all_products
    
    def get_final_summary(self, filename: str = "crawled_products.json"):
        """Generate summary statistics from JSON file."""
        if not os.path.exists(filename):
            print("[ERROR] File does not exist!")
            return
        
        with open(filename, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        categories = {}
        brands = {}
        total_value = 0
        in_stock = 0
        
        for p in data:
            cat = p.get('category', 'Other')
            categories[cat] = categories.get(cat, 0) + 1
            
            brand = p.get('brand', '')
            if brand:
                brands[brand] = brands.get(brand, 0) + 1
            
            total_value += p.get('price', 0)
            if p.get('in_stock', True):
                in_stock += 1
        
        print(f"""
{'=' * 60}
SUMMARY - {filename}
{'=' * 60}

TOTAL PRODUCTS: {len(data):,}
   In stock: {in_stock:,}
   Out of stock: {len(data) - in_stock:,}

VALUE:
   Total: {total_value:,}d
   Average: {total_value // len(data) if data else 0:,}d

BY CATEGORY ({len(categories)} categories):
{'=' * 40}""")
        
        for cat, count in sorted(categories.items(), key=lambda x: -x[1]):
            print(f"   - {cat}: {count:,} products")
        
        print(f"\nTOP 15 BRANDS:\n{'=' * 40}")
        for brand, count in sorted(brands.items(), key=lambda x: -x[1])[:15]:
            if brand:
                print(f"   - {brand}: {count:,} products")


if __name__ == "__main__":
    
    crawler = GearVNCrawlerPro(
        base_url="https://gearvn.com",
        headless=True
    )
    
    selected_categories = [
        ("collections/laptop-gaming-ban-chay", "Laptop Gaming"),
        ("collections/laptop-van-phong", "Laptop Van Phong"),
        ("collections/pc-gvn", "PC GVN"),
        ("collections/man-hinh-may-tinh", "Man Hinh"),
        ("collections/ban-phim-may-tinh", "Ban Phim"),
        ("collections/chuot-may-tinh", "Chuot"),
        ("collections/tai-nghe-may-tinh", "Tai Nghe"),
        ("collections/vga-card-man-hinh", "VGA Card"),
    ]
    
    data = crawler.crawl_all(
        categories=selected_categories,
        save_every=15,
        filename="crawled_products.json"
    )
    
    crawler.get_final_summary("crawled_products.json")


GEARVN CRAWLER PRO - CRAWL ALL PRODUCTS

Categories: 8
Output file: crawled_products.json
Started: 2026-01-22 18:14:59


[1/8] Laptop Gaming

CRAWL CATEGORY: Laptop Gaming
  [Page 1] https://gearvn.com/collections/laptop-gaming-ban-chay
     [+] Found 49 new products
  [Page 2] https://gearvn.com/collections/laptop-gaming-ban-chay?page=2
     [+] Found 49 new products
  [*] Reached last page (2 pages)
  [TOTAL] 98 product links from Laptop Gaming
  [1/98] laptop-gaming-acer-nitro-v-15-anv15-41-r...
      [+] Laptop gaming Acer Nitro V 15 ANV15... | 19,690,000d
  [2/98] laptop-gaming-gigabyte-a16-cmhh2vn893sh...
      [+] Laptop gaming Gigabyte A16 CMHH2VN8... | 22,190,000d
  [3/98] laptop-gaming-asus-rog-zephyrus-g14-ga40...
      [+] Laptop gaming ASUS ROG Zephyrus G14... | 66,890,000d
  [4/98] laptop-gaming-msi-katana-15-hx-b14wek-02...
      [+] Laptop gaming MSI Katana 15 HX B14W... | 33,990,000d
  [5/98] may-choi-game-cam-tay-lenovo-legion-go...
      [+] Máy chơi Game cầm tay Le

In [5]:
"""JSON utility functions for deduplication, merging, and statistics."""

import json
import os
from typing import Optional


def deduplicate_json(
    filename: str = "crawled_products.json", 
    output_filename: Optional[str] = None
) -> int:
    """
    Remove duplicate products from JSON file.
    
    Args:
        filename: JSON file to process
        output_filename: Output file (None = overwrite original)
    
    Returns:
        Number of products after deduplication
    """
    if not os.path.exists(filename):
        print(f"[ERROR] File {filename} does not exist!")
        return 0
    
    print(f"\n{'=' * 60}")
    print(f"DEDUPLICATING FILE: {filename}")
    print('=' * 60)
    
    with open(filename, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    original_count = len(data)
    print(f"Original count: {original_count:,}")
    
    # Filter by URL (unique)
    seen_urls = set()
    unique_products = []
    url_duplicates = 0
    
    for product in data:
        url = product.get('url', '')
        if url and url not in seen_urls:
            seen_urls.add(url)
            unique_products.append(product)
        else:
            url_duplicates += 1
    
    # Filter by name + price (same product with different URL)
    seen_name_price = set()
    final_products = []
    name_duplicates = 0
    
    for product in unique_products:
        name = product.get('name', '').strip().lower()
        price = product.get('price', 0)
        key = f"{name}_{price}"
        
        if key not in seen_name_price:
            seen_name_price.add(key)
            final_products.append(product)
        else:
            name_duplicates += 1
    
    final_count = len(final_products)
    total_removed = original_count - final_count
    
    print(f"\nRESULTS:")
    print(f"   - URL duplicates: {url_duplicates:,}")
    print(f"   - Name+price duplicates: {name_duplicates:,}")
    print(f"   - Total removed: {total_removed:,}")
    print(f"   - Remaining: {final_count:,}")
    
    output = output_filename if output_filename else filename
    with open(output, 'w', encoding='utf-8') as f:
        json.dump(final_products, f, ensure_ascii=False, indent=2)
    
    print(f"\n[SAVED] {output}")
    print('=' * 60 + "\n")
    
    return final_count


def merge_json_files(
    file_list: list, 
    output_filename: str = "merged_products.json", 
    deduplicate: bool = True
) -> int:
    """
    Merge multiple JSON files into one with optional deduplication.
    
    Args:
        file_list: List of JSON files to merge
        output_filename: Output file
        deduplicate: Whether to remove duplicates
    
    Returns:
        Final product count
    """
    print(f"\n{'=' * 60}")
    print("MERGING JSON FILES")
    print('=' * 60)
    
    all_products = []
    
    for filepath in file_list:
        if os.path.exists(filepath):
            with open(filepath, 'r', encoding='utf-8') as f:
                data = json.load(f)
                all_products.extend(data)
                print(f"   [+] {filepath}: {len(data):,} products")
        else:
            print(f"   [-] {filepath}: Not found")
    
    print(f"\nTotal before dedup: {len(all_products):,}")
    
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(all_products, f, ensure_ascii=False, indent=2)
    
    if deduplicate:
        final_count = deduplicate_json(output_filename)
    else:
        final_count = len(all_products)
    
    return final_count


def load_existing_urls(filename: str = "crawled_products.json") -> set:
    """
    Load crawled URLs from file to avoid re-crawling.
    
    Args:
        filename: JSON file with crawled data
    
    Returns:
        Set of crawled URLs
    """
    if not os.path.exists(filename):
        return set()
    
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            data = json.load(f)
        urls = {p.get('url', '') for p in data if p.get('url')}
        print(f"[LOADED] {len(urls):,} URLs from {filename}")
        return urls
    except Exception:
        return set()


def get_stats(filename: str = "crawled_products.json"):
    """Generate detailed statistics from JSON file."""
    if not os.path.exists(filename):
        print(f"[ERROR] File {filename} does not exist!")
        return
    
    with open(filename, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Check duplicates
    urls = [p.get('url', '') for p in data]
    unique_urls = set(urls)
    duplicates = len(urls) - len(unique_urls)
    
    # Statistics
    categories = {}
    brands = {}
    total_value = 0
    in_stock = 0
    has_specs = 0
    has_images = 0
    
    for p in data:
        cat = p.get('category', 'Other')
        categories[cat] = categories.get(cat, 0) + 1
        
        brand = p.get('brand', '')
        if brand:
            brands[brand] = brands.get(brand, 0) + 1
        
        total_value += p.get('price', 0)
        if p.get('in_stock', True):
            in_stock += 1
        if p.get('specs'):
            has_specs += 1
        if p.get('images'):
            has_images += 1
    
    file_size = os.path.getsize(filename) / 1024
    
    print(f"""
{'=' * 60}
DETAILED STATISTICS - {filename}
{'=' * 60}

FILE INFO:
   - Size: {file_size:.2f} KB
   - Products: {len(data):,}
   - URL duplicates: {duplicates:,}

STATUS:
   - In stock: {in_stock:,} ({in_stock * 100 // len(data) if data else 0}%)
   - Out of stock: {len(data) - in_stock:,}
   - Has specs: {has_specs:,} ({has_specs * 100 // len(data) if data else 0}%)
   - Has images: {has_images:,} ({has_images * 100 // len(data) if data else 0}%)

VALUE:
   - Total: {total_value:,}d
   - Average: {total_value // len(data) if data else 0:,}d

BY CATEGORY ({len(categories)}):
{'=' * 40}""")
    
    for cat, count in sorted(categories.items(), key=lambda x: -x[1]):
        pct = count * 100 // len(data)
        bar = "#" * (pct // 5) + "-" * (20 - pct // 5)
        print(f"   {bar} {cat}: {count:,} ({pct}%)")
    
    print(f"\nTOP 10 BRANDS:\n{'=' * 40}")
    for brand, count in sorted(brands.items(), key=lambda x: -x[1])[:10]:
        if brand:
            print(f"   - {brand}: {count:,} products")
    
    if duplicates > 0:
        print(f"\n[!] Found {duplicates} duplicates. Run deduplicate_json('{filename}') to clean.")


# Usage
print("UTILITY FUNCTIONS LOADED:")
print("   - deduplicate_json(filename) - Remove duplicates")
print("   - merge_json_files([file1, file2, ...]) - Merge files")
print("   - load_existing_urls(filename) - Load crawled URLs")
print("   - get_stats(filename) - Detailed statistics")

if os.path.exists("crawled_products.json"):
    get_stats("crawled_products.json")

UTILITY FUNCTIONS LOADED:
   - deduplicate_json(filename) - Remove duplicates
   - merge_json_files([file1, file2, ...]) - Merge files
   - load_existing_urls(filename) - Load crawled URLs
   - get_stats(filename) - Detailed statistics

DETAILED STATISTICS - crawled_products.json

FILE INFO:
   - Size: 435.82 KB
   - Products: 380
   - URL duplicates: 0

STATUS:
   - In stock: 380 (100%)
   - Out of stock: 0
   - Has specs: 246 (64%)
   - Has images: 13 (3%)

VALUE:
   - Total: 6,536,625,000d
   - Average: 17,201,644d

BY CATEGORY (8):
   ##------------------ Tai Nghe: 53 (13%)
   ##------------------ Man Hinh: 51 (13%)
   ##------------------ Ban Phim: 51 (13%)
   ##------------------ Chuot: 50 (13%)
   ##------------------ VGA Card: 50 (13%)
   ##------------------ Laptop Gaming: 49 (12%)
   ##------------------ PC GVN: 40 (10%)
   #------------------- Laptop Van Phong: 36 (9%)

TOP 10 BRANDS:
   - ASUS: 62 products
   - Acer: 44 products
   - Logitech: 39 products
   - MSI: 38 produ

# 📊 Data Cleaning & Database Integration

## Phân tích cấu trúc dữ liệu:

### 1. **Bảng `products`** (Database chính)
Lưu thông tin sản phẩm cơ bản cho e-commerce:
- `sku`, `name`, `price`, `description`, `image_url`, `category_id`

### 2. **Bảng `kb_articles`** (Knowledge Base cho RAG)
Lưu nội dung chi tiết để chatbot tư vấn:
- `title`, `body_md` (Markdown content với specs, mô tả chi tiết)

## Đề xuất:
**Dữ liệu crawled nên được lưu vào CẢ HAI bảng:**

| Mục đích | Bảng | Lý do |
|----------|------|-------|
| Bán hàng, tìm kiếm, đặt hàng | `products` | Thông tin cơ bản: tên, giá, SKU, tồn kho |
| Chatbot RAG tư vấn | `kb_articles` | Nội dung chi tiết: specs, mô tả, so sánh sản phẩm |

**Workflow:**
1. Clean data → `products` (thông tin cơ bản)
2. Clean data → `kb_articles` (nội dung Markdown để vector hóa)
3. Index `kb_articles` vào ChromaDB cho semantic search

In [8]:
"""
Data Cleaning & Transformation Script
Chuẩn bị dữ liệu cho:
1. Bảng products (e-commerce)
2. Bảng kb_articles (Knowledge Base cho RAG chatbot)
"""

import json
import os
import re
import hashlib
from datetime import datetime
from typing import List, Dict, Any, Optional
import uuid


class ProductDataCleaner:
    """Clean và transform dữ liệu crawled cho database."""
    
    def __init__(self, input_file: str = "crawled_products.json"):
        self.input_file = input_file
        self.raw_data = []
        self.cleaned_products = []
        self.kb_articles = []
        
    def load_data(self) -> int:
        """Load raw data từ JSON file."""
        if not os.path.exists(self.input_file):
            print(f"[ERROR] File {self.input_file} không tồn tại!")
            return 0
        
        with open(self.input_file, 'r', encoding='utf-8') as f:
            self.raw_data = json.load(f)
        
        print(f"[LOADED] {len(self.raw_data):,} sản phẩm từ {self.input_file}")
        return len(self.raw_data)
    
    def generate_sku(self, name: str, category: str, url: str) -> str:
        """Generate SKU từ tên sản phẩm."""
        # Lấy mã từ URL (phần cuối)
        url_slug = url.split("/")[-1] if url else ""
        
        # Tạo prefix từ category
        category_prefixes = {
            "Laptop Gaming": "LPG",
            "Laptop Van Phong": "LPV",
            "PC GVN": "PCG",
            "Man Hinh": "MNH",
            "Ban Phim": "BPH",
            "Chuot": "CHU",
            "Tai Nghe": "TNG",
            "Ghe Gaming": "GHE",
            "VGA Card": "VGA",
            "CPU": "CPU",
            "Mainboard": "MBD",
            "RAM": "RAM",
            "O Cung SSD": "SSD",
            "Case": "CAS",
            "Nguon": "PSU",
            "Tan Nhiet": "CLR",
            "Webcam": "WBC",
            "Loa": "LOA",
            "Phu Kien": "PKI",
            "Console/Handheld": "CNS",
        }
        prefix = category_prefixes.get(category, "PRD")
        
        # Hash để tạo unique ID
        hash_str = hashlib.md5(url.encode()).hexdigest()[:6].upper()
        
        return f"{prefix}-{hash_str}"
    
    def clean_price(self, price: Any) -> float:
        """Clean và validate giá."""
        try:
            price_val = float(price)
            return round(price_val, 2) if price_val > 0 else 0.0
        except (ValueError, TypeError):
            return 0.0
    
    def clean_text(self, text: str, max_length: int = None) -> str:
        """Clean text: remove extra whitespace, normalize."""
        if not text:
            return ""
        
        # Remove extra whitespace
        cleaned = " ".join(text.split())
        
        # Truncate if needed
        if max_length and len(cleaned) > max_length:
            cleaned = cleaned[:max_length - 3] + "..."
        
        return cleaned
    
    def specs_to_markdown(self, specs: Dict[str, str], name: str, price: float, category: str) -> str:
        """Convert specs dict to Markdown format cho kb_articles."""
        lines = [
            f"# {name}",
            f"",
            f"**Danh mục:** {category}",
            f"**Giá:** {price:,.0f}đ",
            f"",
            f"## Thông số kỹ thuật",
            f""
        ]
        
        for key, value in specs.items():
            if key and value:
                lines.append(f"- **{key}:** {value}")
        
        return "\n".join(lines)
    
    def extract_brand(self, name: str, existing_brand: str) -> str:
        """Extract hoặc validate brand từ tên sản phẩm."""
        if existing_brand and existing_brand.strip():
            return existing_brand.strip()
        
        # Danh sách brands phổ biến
        known_brands = [
            'ASUS', 'MSI', 'Lenovo', 'Acer', 'HP', 'Dell', 'Gigabyte', 'GIGABYTE',
            'Razer', 'Logitech', 'Corsair', 'HyperX', 'SteelSeries', 'Kingston',
            'Samsung', 'LG', 'BenQ', 'ViewSonic', 'AOC', 'Intel', 'AMD',
            'Cooler Master', 'NZXT', 'Thermaltake', 'Deepcool', 'Noctua',
            'Western Digital', 'WD', 'Seagate', 'Crucial', 'G.Skill',
            'NVIDIA', 'Zotac', 'Galax', 'Colorful', 'Palit',
            'Akko', 'Leopold', 'Ducky', 'Keychron', 'Fuhlen',
            'DareU', 'E-DRA', 'Rapoo', 'Microlab', 'Edifier'
        ]
        
        name_upper = name.upper()
        for brand in known_brands:
            if brand.upper() in name_upper:
                return brand
        
        return ""
    
    def clean_for_products_table(self, product: Dict) -> Dict:
        """
        Transform dữ liệu cho bảng products.
        
        Schema: id, sku, name, price, description, image_url, category_id, created_at, updated_at
        """
        name = self.clean_text(product.get("name", "Unknown"), max_length=255)
        price = self.clean_price(product.get("price", 0))
        category = product.get("category", "Other")
        url = product.get("url", "")
        
        # Generate SKU
        sku = self.generate_sku(name, category, url)
        
        # Get first image or None
        images = product.get("images", [])
        image_url = images[0] if images else None
        
        # Create description từ specs
        specs = product.get("specs", {})
        description_parts = [product.get("description", "")]
        
        # Add key specs to description
        key_specs = ["CPU", "RAM", "Card đồ họa", "Màn hình", "Ổ cứng"]
        for spec_key in key_specs:
            if spec_key in specs:
                description_parts.append(f"{spec_key}: {specs[spec_key]}")
        
        description = " | ".join([p for p in description_parts if p])[:1000]
        
        return {
            "id": str(uuid.uuid4()),
            "sku": sku,
            "name": name,
            "price": price,
            "description": description,
            "image_url": image_url,
            "category": category,  # Sẽ cần map sang category_id sau
            "brand": self.extract_brand(name, product.get("brand", "")),
            "in_stock": product.get("in_stock", True),
            "original_price": self.clean_price(product.get("original_price", 0)),
            "discount_percent": product.get("discount_percent", 0),
            "source_url": url,
            "created_at": datetime.now().isoformat(),
            "updated_at": datetime.now().isoformat()
        }
    
    def clean_for_kb_articles_table(self, product: Dict) -> Dict:
        """
        Transform dữ liệu cho bảng kb_articles (Knowledge Base).
        
        Schema: id, code, title, body_md, created_by, created_at, updated_at, is_public
        """
        name = self.clean_text(product.get("name", "Unknown"), max_length=255)
        price = self.clean_price(product.get("price", 0))
        category = product.get("category", "Other")
        specs = product.get("specs", {})
        url = product.get("url", "")
        brand = self.extract_brand(name, product.get("brand", ""))
        
        # Generate unique code
        hash_str = hashlib.md5(url.encode()).hexdigest()[:8].upper()
        code = f"PROD-{hash_str}"
        
        # Create rich Markdown content for RAG
        body_lines = [
            f"# {name}",
            f"",
            f"## Thông tin cơ bản",
            f"- **Thương hiệu:** {brand if brand else 'Chưa xác định'}",
            f"- **Danh mục:** {category}",
            f"- **Giá bán:** {price:,.0f}đ",
        ]
        
        original_price = product.get("original_price", 0)
        discount = product.get("discount_percent", 0)
        if original_price > price:
            body_lines.append(f"- **Giá gốc:** {original_price:,.0f}đ")
            body_lines.append(f"- **Giảm giá:** {discount}%")
        
        in_stock = product.get("in_stock", True)
        body_lines.append(f"- **Tình trạng:** {'Còn hàng' if in_stock else 'Hết hàng'}")
        
        # Add specs section
        if specs:
            body_lines.extend([
                f"",
                f"## Thông số kỹ thuật chi tiết",
                f""
            ])
            
            for key, value in specs.items():
                if key and value:
                    # Clean value
                    clean_value = self.clean_text(str(value), max_length=500)
                    body_lines.append(f"### {key}")
                    body_lines.append(f"{clean_value}")
                    body_lines.append("")
        
        # Add description
        description = product.get("description", "")
        if description:
            body_lines.extend([
                f"## Mô tả sản phẩm",
                f"{self.clean_text(description, max_length=2000)}",
                f""
            ])
        
        # Source link removed - để trống theo yêu cầu
        
        body_md = "\n".join(body_lines)
        
        return {
            "id": str(uuid.uuid4()),
            "code": code,
            "title": f"[{category}] {name}",
            "body_md": body_md,
            "created_by": None,  # System generated
            "created_at": datetime.now().isoformat(),
            "updated_at": datetime.now().isoformat(),
            "is_public": True,
            # Metadata for later use
            "_meta": {
                "category": category,
                "brand": brand,
                "price": price,
                "source_url": ""  # Để trống theo yêu cầu
            }
        }
    
    def process_all(self) -> Dict[str, int]:
        """
        Process tất cả dữ liệu và tạo 2 output files.
        """
        if not self.raw_data:
            self.load_data()
        
        print(f"\n{'=' * 60}")
        print("PROCESSING DATA FOR DATABASE")
        print('=' * 60)
        
        self.cleaned_products = []
        self.kb_articles = []
        
        seen_skus = set()
        seen_codes = set()
        
        for i, product in enumerate(self.raw_data):
            # Clean for products table
            cleaned = self.clean_for_products_table(product)
            
            # Avoid duplicate SKUs
            if cleaned["sku"] not in seen_skus:
                seen_skus.add(cleaned["sku"])
                self.cleaned_products.append(cleaned)
            
            # Clean for kb_articles table
            kb_article = self.clean_for_kb_articles_table(product)
            
            # Avoid duplicate codes
            if kb_article["code"] not in seen_codes:
                seen_codes.add(kb_article["code"])
                self.kb_articles.append(kb_article)
            
            if (i + 1) % 100 == 0:
                print(f"  Processed {i + 1:,}/{len(self.raw_data):,} products...")
        
        return {
            "products": len(self.cleaned_products),
            "kb_articles": len(self.kb_articles)
        }
    
    def save_outputs(
        self, 
        products_file: str = "cleaned_products.json",
        kb_file: str = "cleaned_kb_articles.json"
    ):
        """Save cleaned data to JSON files."""
        
        # Save products
        with open(products_file, 'w', encoding='utf-8') as f:
            json.dump(self.cleaned_products, f, ensure_ascii=False, indent=2)
        print(f"\n[SAVED] {len(self.cleaned_products):,} products → {products_file}")
        
        # Save KB articles
        with open(kb_file, 'w', encoding='utf-8') as f:
            json.dump(self.kb_articles, f, ensure_ascii=False, indent=2)
        print(f"[SAVED] {len(self.kb_articles):,} KB articles → {kb_file}")
        
        # Generate summary
        self._print_summary()
    
    def _print_summary(self):
        """Print summary statistics."""
        
        # Category distribution
        categories = {}
        brands = {}
        price_ranges = {
            "Dưới 5 triệu": 0,
            "5-10 triệu": 0,
            "10-20 triệu": 0,
            "20-50 triệu": 0,
            "Trên 50 triệu": 0
        }
        
        for p in self.cleaned_products:
            cat = p.get("category", "Other")
            categories[cat] = categories.get(cat, 0) + 1
            
            brand = p.get("brand", "")
            if brand:
                brands[brand] = brands.get(brand, 0) + 1
            
            price = p.get("price", 0)
            if price < 5_000_000:
                price_ranges["Dưới 5 triệu"] += 1
            elif price < 10_000_000:
                price_ranges["5-10 triệu"] += 1
            elif price < 20_000_000:
                price_ranges["10-20 triệu"] += 1
            elif price < 50_000_000:
                price_ranges["20-50 triệu"] += 1
            else:
                price_ranges["Trên 50 triệu"] += 1
        
        print(f"""
{'=' * 60}
📊 DATA CLEANING SUMMARY
{'=' * 60}

✅ PRODUCTS TABLE: {len(self.cleaned_products):,} records
   (For e-commerce: cart, orders, inventory)

✅ KB_ARTICLES TABLE: {len(self.kb_articles):,} records
   (For RAG chatbot: product consultation, recommendations)

📂 BY CATEGORY ({len(categories)} categories):
{'=' * 40}""")
        
        for cat, count in sorted(categories.items(), key=lambda x: -x[1]):
            pct = count * 100 // len(self.cleaned_products)
            print(f"   {cat}: {count:,} ({pct}%)")
        
        print(f"""
💰 BY PRICE RANGE:
{'=' * 40}""")
        for pr, count in price_ranges.items():
            pct = count * 100 // len(self.cleaned_products) if self.cleaned_products else 0
            print(f"   {pr}: {count:,} ({pct}%)")
        
        print(f"""
🏷️ TOP 10 BRANDS:
{'=' * 40}""")
        for brand, count in sorted(brands.items(), key=lambda x: -x[1])[:10]:
            print(f"   {brand}: {count:,}")


# === EXECUTE CLEANING ===
print("=" * 60)
print("🧹 PRODUCT DATA CLEANING & TRANSFORMATION")
print("=" * 60)

cleaner = ProductDataCleaner("crawled_products.json")
result = cleaner.process_all()
cleaner.save_outputs(
    products_file="cleaned_products.json",
    kb_file="cleaned_kb_articles.json"
)

🧹 PRODUCT DATA CLEANING & TRANSFORMATION
[LOADED] 380 sản phẩm từ crawled_products.json

PROCESSING DATA FOR DATABASE
  Processed 100/380 products...
  Processed 200/380 products...
  Processed 300/380 products...

[SAVED] 380 products → cleaned_products.json
[SAVED] 380 KB articles → cleaned_kb_articles.json

📊 DATA CLEANING SUMMARY

✅ PRODUCTS TABLE: 380 records
   (For e-commerce: cart, orders, inventory)

✅ KB_ARTICLES TABLE: 380 records
   (For RAG chatbot: product consultation, recommendations)

📂 BY CATEGORY (8 categories):
   Tai Nghe: 53 (13%)
   Man Hinh: 51 (13%)
   Ban Phim: 51 (13%)
   Chuot: 50 (13%)
   VGA Card: 50 (13%)
   Laptop Gaming: 49 (12%)
   PC GVN: 40 (10%)
   Laptop Van Phong: 36 (9%)

💰 BY PRICE RANGE:
   Dưới 5 triệu: 183 (48%)
   5-10 triệu: 43 (11%)
   10-20 triệu: 51 (13%)
   20-50 triệu: 74 (19%)
   Trên 50 triệu: 29 (7%)

🏷️ TOP 10 BRANDS:
   ASUS: 62
   Acer: 44
   Logitech: 39
   MSI: 38
   Intel: 25
   Gigabyte: 20
   Razer: 20
   Lenovo: 14
   DareU

In [7]:
"""
Database Import Script
Import dữ liệu đã clean vào MySQL database.
"""

import json
import os
import sys

# Add project root to path
project_root = os.path.dirname(os.path.dirname(os.path.abspath(".")))
if project_root not in sys.path:
    sys.path.insert(0, project_root)


def import_to_products_table(db_session, products_data: list):
    """
    Import cleaned products vào bảng products.
    """
    from sqlalchemy import text
    
    imported = 0
    skipped = 0
    
    for product in products_data:
        try:
            # Check if SKU exists
            existing = db_session.execute(
                text("SELECT id FROM products WHERE sku = :sku"),
                {"sku": product["sku"]}
            ).fetchone()
            
            if existing:
                skipped += 1
                continue
            
            # Get or create category
            category_id = None
            category_name = product.get("category", "Other")
            
            cat_result = db_session.execute(
                text("SELECT id FROM categories WHERE name = :name"),
                {"name": category_name}
            ).fetchone()
            
            if cat_result:
                category_id = cat_result[0]
            else:
                # Create category
                import uuid
                category_id = str(uuid.uuid4())
                category_code = category_name.upper().replace(" ", "_")[:64]
                db_session.execute(
                    text("""
                        INSERT INTO categories (id, code, name) 
                        VALUES (:id, :code, :name)
                    """),
                    {"id": category_id, "code": category_code, "name": category_name}
                )
            
            # Insert product
            db_session.execute(
                text("""
                    INSERT INTO products (id, sku, name, price, description, image_url, category_id)
                    VALUES (:id, :sku, :name, :price, :description, :image_url, :category_id)
                """),
                {
                    "id": product["id"],
                    "sku": product["sku"],
                    "name": product["name"],
                    "price": product["price"],
                    "description": product.get("description", ""),
                    "image_url": product.get("image_url"),
                    "category_id": category_id
                }
            )
            imported += 1
            
        except Exception as e:
            print(f"[ERROR] Product {product.get('sku', 'unknown')}: {e}")
            skipped += 1
    
    db_session.commit()
    return {"imported": imported, "skipped": skipped}


def import_to_kb_articles_table(db_session, kb_data: list):
    """
    Import cleaned KB articles vào bảng kb_articles.
    """
    from sqlalchemy import text
    
    imported = 0
    skipped = 0
    
    for article in kb_data:
        try:
            # Check if code exists
            existing = db_session.execute(
                text("SELECT id FROM kb_articles WHERE code = :code"),
                {"code": article["code"]}
            ).fetchone()
            
            if existing:
                skipped += 1
                continue
            
            # Insert KB article
            db_session.execute(
                text("""
                    INSERT INTO kb_articles (id, code, title, body_md, is_public)
                    VALUES (:id, :code, :title, :body_md, :is_public)
                """),
                {
                    "id": article["id"],
                    "code": article["code"],
                    "title": article["title"],
                    "body_md": article["body_md"],
                    "is_public": article.get("is_public", True)
                }
            )
            imported += 1
            
        except Exception as e:
            print(f"[ERROR] KB Article {article.get('code', 'unknown')}: {e}")
            skipped += 1
    
    db_session.commit()
    return {"imported": imported, "skipped": skipped}


def index_kb_to_chromadb(rag_pipeline, kb_data: list):
    """
    Index KB articles vào ChromaDB cho semantic search.
    """
    total_chunks = 0
    
    for i, article in enumerate(kb_data):
        try:
            chunks = rag_pipeline.index_knowledge_article(
                article_id=article["id"],
                title=article["title"],
                content=article["body_md"],
                category=article.get("_meta", {}).get("category", "Product")
            )
            total_chunks += chunks
            
            if (i + 1) % 50 == 0:
                print(f"  Indexed {i + 1}/{len(kb_data)} articles ({total_chunks} chunks)")
                
        except Exception as e:
            print(f"[ERROR] Indexing {article.get('code', 'unknown')}: {e}")
    
    return total_chunks


# ===== DEMO: Load cleaned data =====
print("=" * 60)
print("📥 LOADING CLEANED DATA")
print("=" * 60)

# Load cleaned products
if os.path.exists("cleaned_products.json"):
    with open("cleaned_products.json", "r", encoding="utf-8") as f:
        cleaned_products = json.load(f)
    print(f"✅ Loaded {len(cleaned_products):,} products từ cleaned_products.json")
else:
    print("❌ File cleaned_products.json chưa tồn tại. Chạy cell trước để tạo.")
    cleaned_products = []

# Load cleaned KB articles
if os.path.exists("cleaned_kb_articles.json"):
    with open("cleaned_kb_articles.json", "r", encoding="utf-8") as f:
        cleaned_kb_articles = json.load(f)
    print(f"✅ Loaded {len(cleaned_kb_articles):,} KB articles từ cleaned_kb_articles.json")
else:
    print("❌ File cleaned_kb_articles.json chưa tồn tại. Chạy cell trước để tạo.")
    cleaned_kb_articles = []

# Show sample data
if cleaned_products:
    print(f"\n📦 SAMPLE PRODUCT (for products table):")
    print("-" * 40)
    sample = cleaned_products[0]
    for key in ["sku", "name", "price", "category", "brand"]:
        print(f"   {key}: {sample.get(key)}")

if cleaned_kb_articles:
    print(f"\n📚 SAMPLE KB ARTICLE (for kb_articles table):")
    print("-" * 40)
    sample = cleaned_kb_articles[0]
    print(f"   code: {sample.get('code')}")
    print(f"   title: {sample.get('title')[:60]}...")
    print(f"   body_md: {len(sample.get('body_md', ''))} characters")

print(f"""
{'=' * 60}
📌 NEXT STEPS TO IMPORT DATA:
{'=' * 60}

1. Đảm bảo database MySQL đang chạy
2. Chạy code sau để import:

   from backend.database.session import SessionLocal
   from ai_modules.rag_pipeline.rag_pipeline import RAGPipeline
   
   db = SessionLocal()
   
   # Import products
   result1 = import_to_products_table(db, cleaned_products)
   print(f"Products: {{result1}}")
   
   # Import KB articles
   result2 = import_to_kb_articles_table(db, cleaned_kb_articles)
   print(f"KB Articles: {{result2}}")
   
   # Index to ChromaDB for RAG
   rag = RAGPipeline()
   chunks = index_kb_to_chromadb(rag, cleaned_kb_articles)
   print(f"ChromaDB: {{chunks}} chunks indexed")
   
   db.close()
""")

📥 LOADING CLEANED DATA
✅ Loaded 380 products từ cleaned_products.json
✅ Loaded 380 KB articles từ cleaned_kb_articles.json

📦 SAMPLE PRODUCT (for products table):
----------------------------------------
   sku: LPG-832041
   name: Laptop gaming Acer Nitro V 15 ANV15 41 R1JY
   price: 19690000.0
   category: Laptop Gaming
   brand: Acer

📚 SAMPLE KB ARTICLE (for kb_articles table):
----------------------------------------
   code: PROD-83204102
   title: [Laptop Gaming] Laptop gaming Acer Nitro V 15 ANV15 41 R1JY...
   body_md: 2074 characters

📌 NEXT STEPS TO IMPORT DATA:

1. Đảm bảo database MySQL đang chạy
2. Chạy code sau để import:

   from backend.database.session import SessionLocal
   from ai_modules.rag_pipeline.rag_pipeline import RAGPipeline

   db = SessionLocal()

   # Import products
   result1 = import_to_products_table(db, cleaned_products)
   print(f"Products: {result1}")

   # Import KB articles
   result2 = import_to_kb_articles_table(db, cleaned_kb_articles)
   prin